In [1]:
import re
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
from geopy.distance import great_circle

In [2]:
origin_df = pd.read_csv('dataset/902_coordinate.csv')

In [3]:
df11 = pd.read_csv('dataset/031_metro20210906.csv')
df22 = pd.read_csv('dataset/031_regional20210906.csv')

df1 = df11[df11['Time']=='1/1/2023']
df2 = df22[df11['Time']=='1/1/2023']



C:\Users\Yummy\AppData\Local\Temp\ipykernel_15288\2056008060.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df22[df11['Time']=='1/1/2023']


In [13]:
df3 = pd.read_csv('dataset/901_traffic.csv')

In [43]:
df4_1 = pd.read_csv('dataset/04_Copy_of_POI_(Fast_food).csv')
df4_2 = pd.read_csv('dataset/04_Copy_of_POI_(Fuel_station).csv')
df4_3 = pd.read_csv('dataset/04_Copy_of_POI_(Tourism).csv',encoding='ISO-8859-1')


In [4]:
# df3 = gpd.read_file('dataset/032_route_elevations_optimised.geojson')

###### step1. 坐标是否处于区域充电最佳位置，距离最近的充电最佳中心点的距离，属于Metro1, Major towns2，Journey enablement3 

In [5]:
df1 = df1[["Latitude", "Longitude","Zone"]]
df1['location_type'] = 1
type_mapping1 = {
    "1 km": 3
}
df1['Zone'] = df1['Zone'].map(type_mapping1)
# df1

In [6]:
df2 = df2[["Latitude", "Longitude","Type","Zone (km)"]]
df2.columns = ["Latitude", "Longitude","Type","Zone"]
type_mapping2 = {
    "Major towns": 2,
    "Journey enablement": 3
}
df2['location_type'] = df2['Type'].map(type_mapping2)
type_mapping3 = {
    "20km": 20,
    "10km": 10
}
df2['Zone'] = df2['Zone'].map(type_mapping3)
df2 = df2.drop(columns=['Type'])
# df2

In [7]:
region_df = pd.concat([df1, df2], ignore_index=True)
region_df

,Latitude,Longitude,Zone,location_type
0,-33.111289,151.140228,3,1
1,-33.860520,151.201580,3,1
2,-33.208801,151.377213,3,1
3,-33.315827,151.519708,3,1
4,-33.884119,151.212262,3,1
...,...,...,...,...
559,-28.260110,153.579743,10,2
560,-28.790837,153.585243,10,2
561,-28.814773,153.285898,10,2
562,-31.435419,152.897660,10,2


In [8]:
# Haversine公式
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 地球半径，单位为千米
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    km = R * c
    return km

In [9]:
def calculate_distance(df1, df2, feature_name1,feature_name2,feature_name3,feature_name4):
    counts = []
    temps = []
    dists = []
    zones = []
    for index1, row1 in df1.iterrows():
        count = []
        min_dist = 1000000
        min_temp = 0
        zone_temp = 0
        lat1, lon1 = row1['Latitude'], row1['Longitude']
        for index2, row2 in df2.iterrows():
            lat2, lon2 = row2['Latitude'], row2['Longitude']
#             zone_str = str(row2['Zone']).replace('km','')
#             zone_str = zone_str.replace(' ','')
            zone = int(row2['Zone'])
            
            distance = haversine(lat1, lon1, lat2, lon2)
            if distance < min_dist:
                min_dist = distance
                min_temp = row2['location_type']
                zone_temp = zone
            if distance < zone:      ### 如果满足条件，则记录对应的MVA和
                count.append(row2['location_type'])
        dists.append(min_dist)    
        if len(count)==0:
            counts.append(min_temp)
            temps.append(0)
            zones.append(zone_temp)
        elif 1 in count:
            counts.append(1)
            temps.append(1)
            zones.append(3)
        elif 2 in count:
            counts.append(2)
            temps.append(1)
            zones.append(10)
        elif 3 in count:
            counts.append(3)
            temps.append(1)
            zones.append(20)
    df_temp = df1.copy()
    df_temp[feature_name4] = zones
    df_temp[feature_name1] = counts
    df_temp[feature_name2] = temps
    df_temp[feature_name3] = dists
    return df_temp

In [11]:
temp1 = calculate_distance(origin_df, region_df, 'location_type', 'if_optimal_zone','dist_to_optimal_point','Zone')

In [12]:
temp1['location_type'] = temp1['location_type'].astype('int64')

# temp1[temp1['location_type']==1].head(40)
temp1

,Latitude,Longitude,Type,Zone,location_type,if_optimal_zone,dist_to_optimal_point
0,-34.508903,144.842785,1,20,3,1,0.539601
1,-34.746497,146.550076,1,20,3,1,0.887491
2,-35.354863,145.725113,1,20,3,1,0.178044
3,-34.639541,143.564708,1,20,3,1,0.370967
4,-33.923735,147.203554,1,20,3,1,1.075135
...,...,...,...,...,...,...,...
1628,-31.909129,152.460275,4,10,2,1,0.420047
1629,-29.056524,152.018188,4,20,3,0,80.424056
1630,-35.130380,147.341831,4,10,2,1,2.188731
1631,-34.843652,148.914140,4,10,2,1,0.945710


###### step2. 获取Zone范围内的交通总量以及是否有重型交通

In [23]:
df3

,Unnamed: 0,station_id,full_name,suburb,post_code,wgs84_latitude,wgs84_longitude,traffic_count,Heavy_traffic
0,0,1001,"Sydney Harbour Bridge, North of Hickson Road",Milsons Point,2061.0,-33.852116,151.210770,22484,1
1,1,1003,"Sydney Harbour Tunnel, North of Cahill Expressway",Sydney,2000.0,-33.860836,151.213791,73903,0
2,2,1004,"Western Distributor, West of Sussex Street",Sydney,2000.0,-33.866180,151.203201,36900,0
3,3,1011,"Cahill Expressway, East of Pitt Street",Sydney,2000.0,-33.861290,151.209854,36418,0
4,4,1323,"Ultimo Road, East of Darling Drive",Haymarket,2000.0,-33.880939,151.202393,10824,0
...,...,...,...,...,...,...,...,...,...
1778,1778,T0497,"Newell Highway, South of Biddon Forest Road",Biddon,2827.0,-31.616983,148.746139,1707,1
1779,1779,T0498,"Newell Highway, North of Oxley Highway",Coonabarabran,2357.0,-31.236635,149.315750,1458,1
1780,1780,T0557,"M2 Motorway, East of Cropley Drive",NaN,NaN,-33.770721,150.986466,35958,0
1781,1781,THSTC,"Pacific Motorway, South of Heritage Street",Chinderah,2487.0,-28.245531,153.546005,18627,1


In [17]:
def calculate_traffic_count(df1, df2, feature_name):
    sums = []
    tags = []
    temp_df = df1.copy()
    for index1, row1 in df1.iterrows():
        total_traffic_count = 0
        heavy_count = 0
        lat1, lon1 = row1['Latitude'], row1['Longitude']
        zone = row1['Zone']
        for index2, row2 in df2.iterrows():
            lat2, lon2 = row2['wgs84_latitude'], row2['wgs84_longitude']
            distance = haversine(lat1, lon1, lat2, lon2)
            if distance < zone:
                total_traffic_count += row2['traffic_count']
                heavy_count += row2['Heavy_traffic']
        sums.append(total_traffic_count)
        if heavy_count!=0:
            tags.append(1)
        else:
            tags.append(0)
    temp_df[feature_name] = sums
    temp_df['Heavy_traffic'] = tags
    return temp_df

In [18]:
temp2 = calculate_traffic_count(temp1,df3,'traffic_count')

In [30]:
temp2[temp2['Heavy_traffic']==0]

,Latitude,Longitude,Type,Zone,location_type,if_optimal_zone,dist_to_optimal_point,traffic_count,Heavy_traffic
53,-32.787817,151.727705,1,3,1,0,10.336147,8112,0
54,-33.310237,151.420585,1,10,2,1,7.620858,286498,0
63,-30.338275,152.712826,1,10,2,0,35.419239,0,0
69,-33.720695,150.429727,1,3,1,1,2.758153,21240,0
79,-33.614302,150.746806,1,3,1,0,6.509312,0,0
...,...,...,...,...,...,...,...,...,...
1613,-34.436615,150.863298,4,3,1,1,1.803298,362285,0
1615,-33.745229,150.610129,4,3,1,1,2.236363,0,0
1620,-33.895588,151.195760,4,3,1,1,0.524214,1470973,0
1626,-33.917263,151.038839,4,3,1,1,1.943347,583474,0


###### step3. 获取距离最近的已有的快速充电桩和目的地充电桩

In [37]:
chargers_df = origin_df[(origin_df["Type"]==1)|(origin_df["Type"]==2)]
chargers_df

,Latitude,Longitude,Type
0,-34.508903,144.842785,1
1,-34.746497,146.550076,1
2,-35.354863,145.725113,1
3,-34.639541,143.564708,1
4,-33.923735,147.203554,1
...,...,...,...
1043,-35.537269,144.966563,2
1044,-31.960975,141.462880,2
1045,-34.451028,150.447443,2
1046,-33.688731,149.564345,2


In [38]:
### 计算每个现有的station与最近的station的距离
def calculate_shortest_distance(df1, df2):
    counts1 = []
    counts2 = []
    temp_df = df1.copy()
    for index1, row1 in df1.iterrows():
        count1 = 10000000
        count2 = 10000000
        lat1, lon1, type1 = row1['Latitude'], row1['Longitude'], row1['Type']
        for index2, row2 in df2.iterrows():
            lat2, lon2, type2 = row2['Latitude'], row2['Longitude'], row2['Type']
            distance = haversine(lat1, lon1, lat2, lon2)
            if distance!=0 and type2 == 1 and distance < count1:
                count1 = distance
            if distance!=0 and type2 == 2 and distance < count2:
                count2 = distance
        counts1.append(count1)
        counts2.append(count2)
    temp_df['Distance to the nearest fast charger'] = counts1
    temp_df['Distance to the nearest destination charger'] = counts2
    return temp_df

In [40]:
temp3 = calculate_shortest_distance(temp2,chargers_df)

In [41]:
temp3

,Latitude,Longitude,Type,Zone,location_type,if_optimal_zone,dist_to_optimal_point,traffic_count,Heavy_traffic,Distance to the nearest fast charger,Distance to the nearest destination charger
0,-34.508903,144.842785,1,20,3,1,0.539601,14283,1,117.914230,0.022555
1,-34.746497,146.550076,1,20,3,1,0.887491,29358,1,83.735856,25.326981
2,-35.354863,145.725113,1,20,3,1,0.178044,15717,1,75.411076,71.534310
3,-34.639541,143.564708,1,20,3,1,0.370967,8572,1,117.914230,47.392221
4,-33.923735,147.203554,1,20,3,1,1.075135,21018,1,65.591754,88.407090
...,...,...,...,...,...,...,...,...,...,...,...
1628,-31.909129,152.460275,4,10,2,1,0.420047,47595,1,3.929844,0.427711
1629,-29.056524,152.018188,4,20,3,0,80.424056,16135,1,0.072353,0.215983
1630,-35.130380,147.341831,4,10,2,1,2.188731,118910,1,0.156988,0.154377
1631,-34.843652,148.914140,4,10,2,1,0.945710,48072,1,0.015156,0.239368


###### step4. 获取兴趣点( Fast_food, Fuel_station, Tourism)

In [50]:
def calculate_POI_count(df1, df2, feature_name):
    counts = []
    temp_df = df1.copy()
    for index1, row1 in df1.iterrows():
        count = 0
        lat1, lon1 = row1['Latitude'], row1['Longitude']
        zone = row1['Zone']
        for index2, row2 in df2.iterrows():
            lat2, lon2 = row2['latitude'], row2['longitude']
            distance = haversine(lat1, lon1, lat2, lon2)
            if distance < zone:
                count += 1
        counts.append(count)
    temp_df[feature_name] = counts
    return temp_df

In [51]:
temp4 = calculate_POI_count(temp3,df4_1,'Fast_food')

In [52]:
temp4

,Latitude,Longitude,Type,Zone,location_type,if_optimal_zone,dist_to_optimal_point,traffic_count,Heavy_traffic,Distance to the nearest fast charger,Distance to the nearest destination charger,Fast_food
0,-34.508903,144.842785,1,20,3,1,0.539601,14283,1,117.914230,0.022555,2
1,-34.746497,146.550076,1,20,3,1,0.887491,29358,1,83.735856,25.326981,0
2,-35.354863,145.725113,1,20,3,1,0.178044,15717,1,75.411076,71.534310,0
3,-34.639541,143.564708,1,20,3,1,0.370967,8572,1,117.914230,47.392221,0
4,-33.923735,147.203554,1,20,3,1,1.075135,21018,1,65.591754,88.407090,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1628,-31.909129,152.460275,4,10,2,1,0.420047,47595,1,3.929844,0.427711,7
1629,-29.056524,152.018188,4,20,3,0,80.424056,16135,1,0.072353,0.215983,4
1630,-35.130380,147.341831,4,10,2,1,2.188731,118910,1,0.156988,0.154377,15
1631,-34.843652,148.914140,4,10,2,1,0.945710,48072,1,0.015156,0.239368,3


In [53]:
temp5 = calculate_POI_count(temp4,df4_2,'Fuel_station')

In [54]:
temp6 = calculate_POI_count(temp5,df4_3,'Tourism')

In [55]:
temp6

,Latitude,Longitude,Type,Zone,location_type,if_optimal_zone,dist_to_optimal_point,traffic_count,Heavy_traffic,Distance to the nearest fast charger,Distance to the nearest destination charger,Fast_food,Fuel_station,Tourism
0,-34.508903,144.842785,1,20,3,1,0.539601,14283,1,117.914230,0.022555,2,4,2
1,-34.746497,146.550076,1,20,3,1,0.887491,29358,1,83.735856,25.326981,0,2,1
2,-35.354863,145.725113,1,20,3,1,0.178044,15717,1,75.411076,71.534310,0,4,0
3,-34.639541,143.564708,1,20,3,1,0.370967,8572,1,117.914230,47.392221,0,5,1
4,-33.923735,147.203554,1,20,3,1,1.075135,21018,1,65.591754,88.407090,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,-31.909129,152.460275,4,10,2,1,0.420047,47595,1,3.929844,0.427711,7,11,3
1629,-29.056524,152.018188,4,20,3,0,80.424056,16135,1,0.072353,0.215983,4,9,3
1630,-35.130380,147.341831,4,10,2,1,2.188731,118910,1,0.156988,0.154377,15,16,6
1631,-34.843652,148.914140,4,10,2,1,0.945710,48072,1,0.015156,0.239368,3,2,1


In [56]:
# temp6.to_csv('dataset/003_dataset.csv', index=False)

###### step5. 坐标的z_str为距离最近的route的z_str，坐标zone内的route线(是否是交通枢纽)

In [ ]:
geo_df = gpd.read_file('dataset/032_route_elevations_optimised.geojson')

In [ ]:
temp6['geometry'] = temp1.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
temp6

In [ ]:
def min_distance(point, lines):
    distances = lines.distance(point)
    sorted_distances = distances.sort_values()
    for index in sorted_distances.index:
        z_str = lines.iloc[index]['Z_STR']
        if pd.notna(z_str):
            return sorted_distances.iloc[0], z_str
    return sorted_distances.iloc[0], None

In [ ]:
temp6[['dist', 'z_str']] = temp6['geometry'].progress_apply(lambda point: min_distance(point, geo_df)).apply(pd.Series)


In [ ]:
temp7 = temp6.drop(columns=['geometry'])

In [ ]:
temp7

In [ ]:
temp7.to_csv('dataset/903_dataset.csv', index=False)

In [ ]:
# gdf1 = gpd.GeoDataFrame(temp1, geometry=gpd.points_from_xy(temp1.Longitude, temp1.Latitude))
# # gdf1.crs = 'EPSG:4326'  # 设置CRS

In [ ]:
# def min_distance(point, gdf2):
#     gdf2['distance'] = gdf2.apply(lambda row: point.distance(row.geometry), axis=1)
#     min_row = gdf2.loc[gdf2['distance'].idxmin()]
#     return pd.Series({'dist': min_row['distance'] / 1000, 'z_str': min_row['Z_STR']})  # 转换为千米

In [ ]:
# gdf1[['dist', 'z_str']] = gdf1.apply(lambda row: min_distance(row.geometry, df3), axis=1)
